In [ ]:
# Main code for analyzing Ferrimagnetism in alternating spin chains
#-----------------------------------------------------------------------

using ITensors,ITensorMPS
import ITensors:op
using PyFormattedStrings
using HDF5

ITensors.disable_warn_order()

In [16]:
include("SiteType_ThreeHalf.jl");       # define Tensor Operations for 3/2
include("Operations_TwoSite.jl");       # define two site operations
include("Operations_FourSite.jl");      # dfine four site operations
include("Calculate_Entanglement.jl");   # calculate entanglement of tensors

In [17]:
# construction of different magon operators at various sites
# wf = wave function in MPS form
# k_list = list of sites where to create magnons
# e.g. k_list = [A,B] creates a magnon at (A,A+1) and (B,B+1)
# split = "no" is default. spilit = "yes" will do a split magnon
#--------------------------------------------------------------------
function MagnonOperation(wf::MPS, k_list; split="no")    
    psi = copy(wf)
    sites = siteinds(psi)

    #magnon creation at sites k,k+1
    for k in k_list
        if isodd(k)
            psi = apply( op( "S+",sites[k]   ), psi )
            psi = apply( op( "S-",sites[k+1] ), psi )
        else
            psi = apply( op( "S-",sites[k]   ), psi )
            psi = apply( op( "S+",sites[k+1] ), psi )
        end   
    end

    #for split magnon
    if length(k_list)==2 && split=="yes"
        m,n = k_list[1]+1, k_list[2]     #loc of desturction
        
        if isodd(m)
            psi = apply( op("S-",sites[m]), psi )
        else
            psi = apply( op("S+",sites[m]), psi )
        end

        if isodd(n)
            psi = apply( op("S-",sites[n]), psi )
        else
            psi = apply( op("S+",sites[n]), psi )
        end
    end

    return normalize!(psi)
    #----------------------------------
end


MagnonOperation (generic function with 1 method)

In [7]:
# construction of the MPO hamiltonian
# sites =  all sites,
# J = hoppings between different spins
# B = coupling to external magnetic field
#------------------------------------------
function Hamiltonian(sites,J,B, type)
    ops = OpSum()
    N = length(sites)

    for k in 1:N
        ops +=  -B,   "Sz",k
    end

    for k in 1:N-1
        ops += J,       "Sz",k,     "Sz",k+1
        ops += J/2,     "S+",k,     "S-",k+1
        ops += J/2,     "S-",k,     "S+",k+1
    end

    if type=="Loop"
        ops += J,       "Sz",N,     "Sz",1
        ops += J/2,     "S+",N,     "S-",1
        ops += J/2,     "S-",N,     "S+",1
    end


    #--------------------------
    return MPO(ops,sites)
end

Hamiltonian (generic function with 1 method)

In [13]:
#define the parameters of the hamiltonian and constuct MPO
#----------------------------------------------------------------------------

nsweeps = 10
cut_off  = [1E-10]   # error cutoff
maxdim  = [50,100,150,200,250,300,400,500,600,800,1000]

N = 10
J = 1
B = 1
type = "Open"       #specify Open/Loop for open and closed chains


sites = siteinds( k->isodd(k) ? "S=1/2" : "S=3/2", N ) ;

#Neel = productMPS( sites, k->isodd(k) ? "Dn" : "+3/2" ) ; 

In [19]:
H = Hamiltonian(sites,J,B, type) ;
energy,psi0 = dmrg( H, random_mps(sites,linkdims=maxdim[1]); nsweeps, maxdim, cutoff=cut_off, ishermitian=true )

After sweep 1 energy=-14.099063482633055  maxlinkdim=50 maxerr=2.85E-04 time=7.710
After sweep 2 energy=-14.09933141090861  maxlinkdim=20 maxerr=9.38E-11 time=0.032
After sweep 3 energy=-14.099331438165642  maxlinkdim=14 maxerr=9.89E-11 time=0.007
After sweep 4 energy=-14.09933143816685  maxlinkdim=14 maxerr=9.88E-11 time=0.007
After sweep 5 energy=-14.099331438166825  maxlinkdim=14 maxerr=9.88E-11 time=0.007
After sweep 6 energy=-14.09933143816686  maxlinkdim=14 maxerr=9.88E-11 time=0.008
After sweep 7 energy=-14.09933143816685  maxlinkdim=14 maxerr=9.88E-11 time=0.006
After sweep 8 energy=-14.09933143816685  maxlinkdim=14 maxerr=9.88E-11 time=0.007
After sweep 9 energy=-14.099331438166857  maxlinkdim=14 maxerr=9.88E-11 time=0.007
After sweep 10 energy=-14.099331438166855  maxlinkdim=14 maxerr=9.88E-11 time=0.007


(-14.099331438166855, MPS
[1] ((dim=2|id=889|"Link,l=1"), (dim=2|id=798|"S=1/2,Site,n=1"))
[2] ((dim=7|id=883|"Link,l=2"), (dim=4|id=68|"S=3/2,Site,n=2"), (dim=2|id=889|"Link,l=1"))
[3] ((dim=2|id=228|"S=1/2,Site,n=3"), (dim=10|id=758|"Link,l=3"), (dim=7|id=883|"Link,l=2"))
[4] ((dim=4|id=209|"S=3/2,Site,n=4"), (dim=14|id=606|"Link,l=4"), (dim=10|id=758|"Link,l=3"))
[5] ((dim=2|id=668|"S=1/2,Site,n=5"), (dim=13|id=210|"Link,l=5"), (dim=14|id=606|"Link,l=4"))
[6] ((dim=4|id=576|"S=3/2,Site,n=6"), (dim=12|id=489|"Link,l=6"), (dim=13|id=210|"Link,l=5"))
[7] ((dim=2|id=909|"S=1/2,Site,n=7"), (dim=10|id=991|"Link,l=7"), (dim=12|id=489|"Link,l=6"))
[8] ((dim=4|id=84|"S=3/2,Site,n=8"), (dim=6|id=587|"Link,l=8"), (dim=10|id=991|"Link,l=7"))
[9] ((dim=2|id=370|"S=1/2,Site,n=9"), (dim=3|id=558|"Link,l=9"), (dim=6|id=587|"Link,l=8"))
[10] ((dim=4|id=34|"S=3/2,Site,n=10"), (dim=3|id=558|"Link,l=9"))
)

In [20]:
Entanglement = [ calc_LogNeg_FourSite(psi0,[1,3+D]) for D in 0:5 ]

6-element Vector{Float64}:
 0.6583426786383081
 0.3914201090682962
 0.2397642441794473
 0.19367729984733098
 0.16116996809035594
 0.14350185648077937